In [1]:
import numpy as np
import pandas as pd
from datetime import datetime,timedelta

from sklearn.model_selection import train_test_split
from scipy.stats import ranksums
from scipy.stats import pearsonr

from sklearn.linear_model import LogisticRegression,LinearRegression

In [2]:
vix_ts=pd.read_csv('VIX_index_timeseries.csv')

vix_ts['DATE'] = pd.to_datetime(vix_ts['DATE'],format='%d/%m/%y')

IOError: File VIX_index_timeseries.csv does not exist

In [ ]:
vix_ts.head()

In [ ]:
rolling_orders = pd.read_csv('rolling_window_ts.csv')

In [ ]:
rolling_orders = rolling_orders[['six','five','four','three','two','one','sunday_beginning']]
rolling_orders['sunday_beginning'] = pd.to_datetime(rolling_orders['sunday_beginning'])
rolling_orders['monday_beginning'] = rolling_orders['sunday_beginning'] + timedelta(days=1)
rolling_orders.head()

In [ ]:
vix_ts_reg_ready = pd.merge(vix_ts,rolling_orders,how='right',left_on='DATE',right_on='monday_beginning')
vix_ts_reg_ready.head()

In [ ]:
vix_ts_reg_ready['VIX_next_wk'] = vix_ts_reg_ready['VIX CLOSE'].shift(-1)
vix_ts_reg_ready = vix_ts_reg_ready.iloc[:-1,:]

vix_ts_reg_ready['VIX CLOSE'] = vix_ts_reg_ready['VIX CLOSE'].astype(float)
vix_ts_reg_ready['VIX_next_wk'] = vix_ts_reg_ready['VIX_next_wk'].astype(float)

vix_ts_reg_ready['VIX_delta'] = vix_ts_reg_ready['VIX_next_wk'] - vix_ts_reg_ready['VIX CLOSE']
vix_ts_reg_ready.head()

In [ ]:
vix_ts_reg_ready.to_csv('test.csv',index=False)

#### Modeling of VIX_delta~one:six predicting direction

In [ ]:
vix_ts_reg_ready['VIX_direction'] = (vix_ts_reg_ready['VIX_delta']>0).astype(int)

x = vix_ts_reg_ready[['six','five','four','three','two','one']]
y_bin = vix_ts_reg_ready['VIX_direction']
y = vix_ts_reg_ready['VIX_delta']

split = np.floor(0.75*vix_ts_reg_ready.shape[0]).astype(int)
print split

x_train = x.iloc[:split,:]
x_test = x.iloc[split:,:]
y_train = y_bin.iloc[:split]
y_train = np.ravel(y_train)
y_test = y_bin.iloc[split:]
y_test = np.ravel(y_test)

test_return = y.iloc[split:]
test_return = np.ravel(test_return)



In [ ]:
lr = LogisticRegression(class_weight='balanced')
lr.fit(x_train,y_train)

In [ ]:
lr.coef_

In [ ]:
preds = lr.predict_proba(x_test)
preds = pd.DataFrame(preds)
preds = np.array(preds[1])
preds

In [ ]:
y_test = pd.DataFrame({'actual_binary':y_test,'actual_ret':test_return,'preds':preds})
y_test.sort_values(by='preds')
y_test.to_csv('testset.csv')

In [ ]:
print pearsonr(y_test['actual_ret'],y_test['preds'])

In [ ]:
np.mean(y_test['actual_binary'])

In [ ]:
print np.count_nonzero(y_test[y_test['preds']<0.5]['actual_ret'])
print np.mean(y_test[y_test['preds']<0.5]['actual_ret'])

In [ ]:
a = np.linspace(0.30,0.5,5)

for pred in a:
    print "predicted return of "+str(pred)
    print 'count is: '+str(np.count_nonzero(y_test[y_test['preds']<pred]['actual_ret']))
    print np.mean(y_test[y_test['preds']<pred]['actual_binary'])
    print np.mean(y_test[y_test['preds']<pred]['actual_ret'])

In [ ]:
a = np.linspace(0.50,0.7,5)

for pred in a:
    print "predicted return of "+str(pred)
    print 'count is: '+str(np.count_nonzero(y_test[y_test['preds']>pred]['actual_ret']))
    print np.mean(y_test[y_test['preds']>pred]['actual_binary'])
    print np.mean(y_test[y_test['preds']>pred]['actual_ret'])

#### Prediction in regression context

In [ ]:
vix_ts_reg_ready['VIX_direction'] = (vix_ts_reg_ready['VIX_delta']>0).astype(int)

x = vix_ts_reg_ready[['six','five','four','three','two','one']]
y_bin = vix_ts_reg_ready['VIX_direction']
y = vix_ts_reg_ready['VIX_delta']

print (vix_ts_reg_ready['VIX_delta'])

split = np.floor(0.75*vix_ts_reg_ready.shape[0]).astype(int)
print split

x_train = x.iloc[:split,:]
x_test = x.iloc[split:,:]
y_train = y.iloc[:split]
y_test = y.iloc[split:]

test_return = y.iloc[split:]

In [ ]:
lr = LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
lr.coef_

In [ ]:
preds = lr.predict(x_test)
preds = pd.DataFrame(preds)
preds

In [ ]:
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True,drop=True)

print y_test.shape
print preds.shape

In [ ]:
df = y_test.join(preds)
df.rename(columns={0:'preds'},inplace=True)
df

In [ ]:
print np.count_nonzero(df[df['preds']<0]['VIX_delta'])
print np.median(df[df['preds']<0]['VIX_delta'])

In [ ]:
print np.count_nonzero(df[df['preds']>0]['VIX_delta'])
print np.median(df[df['preds']>0]['VIX_delta'])

In [ ]:
pearsonr(df['VIX_delta'],df['preds'])

In [ ]:
third = np.floor(df.shape[0]/3).astype(int)
third

In [ ]:
PREDICTOR = 'VIX_delta'

In [ ]:
y_test1 = df.sort_values(by='preds')
y_test2 = y_test1.reset_index(drop=True)
y_test2

In [ ]:
print np.mean(y_test2.iloc[0:third][PREDICTOR])
print np.mean(y_test2.iloc[third:2*third][PREDICTOR])
print np.mean(y_test2.iloc[2*third:][PREDICTOR])
print "and now for medians:"
print np.median(y_test2.iloc[0:third][PREDICTOR])
print np.median(y_test2.iloc[third:2*third][PREDICTOR])
print np.median(y_test2.iloc[2*third:][PREDICTOR])